<a href="https://colab.research.google.com/github/Nikil263/Spoiler-detection-in-unsolicited-movie-review-in-tweets/blob/main/spoiler_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Spoiler detection in unsolicited movie review in tweets




Creation of dataset

In [4]:
pip install snscrape 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 KB 7.3 MB/s eta 0:00:00


In [5]:
import snscrape.modules.twitter as twt
import pandas as pd


In [ ]:
tw=[]
query= "divergent"
limit=0
for tweet in twt.TwitterSearchScraper(query).get_items():
  if limit>=1000:
    break
  else:
    if tweet.lang=='en':
       limit=limit+1
       tw.append(tweet.content)

query= "source code"
limit=0
for tweet in twt.TwitterSearchScraper(query).get_items():
  if limit>=1000:
    break
  else:
    if tweet.lang=='en':
       limit=limit+1
       tw.append(tweet.content)

query= "book thief"
limit=0
for tweet in twt.TwitterSearchScraper(query).get_items():
  if limit>=1000:
    break
  else:
    if tweet.lang=='en':
       limit=limit+1
       tw.append(tweet.content)
       

query= "hugo"
limit=0
for tweet in twt.TwitterSearchScraper(query).get_items():
  if limit>=1000:
    break
  else:
    if tweet.lang=='en':
       limit=limit+1
       tw.append(tweet.content)

df=pd.DataFrame(tw,columns=['tweet'])

<ipython-input-3-b0e9230d56c3>:10: FutureWarning: content is deprecated, use rawContent instead
  tw.append(tweet.content)
<ipython-input-3-b0e9230d56c3>:20: FutureWarning: content is deprecated, use rawContent instead
  tw.append(tweet.content)
<ipython-input-3-b0e9230d56c3>:30: FutureWarning: content is deprecated, use rawContent instead
  tw.append(tweet.content)
<ipython-input-3-b0e9230d56c3>:41: FutureWarning: content is deprecated, use rawContent instead
  tw.append(tweet.content)


In [ ]:
df['label']=''
df.to_csv('dataset.csv', index=False)
  

Dataset is manually labeled and saved as labeldataset.csv

In [ ]:
df1=pd.read_csv('/labeldataset.csv')
print(df1.shape)

(4000, 2)


Preprocessing the tweets

In [13]:
pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=869e1a22b8d7f607975b8a7520aa3c53f0e34ee0b2b7fdb2a19e7f586052f5fc
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [14]:
import re 
import string
import nltk
import emoji
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [15]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
def preprocess(tweet):
  tweet = emoji.replace_emoji(tweet,'')
  tweet = re.sub(r"http\S+|www\S+|https\S+|@\S+|#\S+", "", tweet, flags=re.MULTILINE)
  tweet = tweet.translate(str.maketrans("", "", string.punctuation))
  tweet = tweet.translate(str.maketrans("", "", string.digits))
  tokens = word_tokenize(tweet)
  tokens = [token.lower() for token in tokens]
  stop_words = set(stopwords.words('english'))
  tokens = [token for token in tokens if token not in stop_words]
  stemmer = PorterStemmer()
  tokens = [stemmer.stem(token) for token in tokens]
  return " ".join(tokens)


Creating preprocesses tweet dataset

In [ ]:
preprocessedtweets = []
for index, row in df1.iterrows():
  preprocessedtweets.append(preprocess(row['TWEET']))
df2 = pd.DataFrame(preprocessedtweets,columns=['tweet'])
df2['label'] = df1['LABEL']
df2['label'] = df2['label'].astype('int')
df2.to_csv('preprocessdataset.csv', index=False)

Splitting dataset to training and validation

In [6]:
df3 = pd.read_csv('/preprocessdataset-2.csv')
train = df3.iloc[:800,:]
val = df3.iloc[800:1001,:]
train1 = df3.iloc[1001:1801,:]
val1 = df3.iloc[1801:2001,:]
train2 = df3.iloc[2001:2801,:]
val2 = df3.iloc[2801:3001,:]
train3 = df3.iloc[3001:3801,:]
val3 = df3.iloc[3801:4001,:]
train = train.append(train1)
train = train.append(train2)
train = train.append(train3)
val = val.append(val1)
val = val.append(val2)
val = val.append(val3)
print(train.shape)
print(val.shape)


(3200, 2)
(800, 2)


Training and Validating the model

In [1]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.4 MB/s eta 0:00:00


In [7]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import numpy as np
import torch

In [8]:
trainl = train['tweet'].tolist()
vall = val['tweet'].tolist()

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=3)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
train_tokens = [tokenizer.encode(str(text), add_special_tokens=True, truncation=True, padding=True) for text in train['tweet']]
val_tokens = [tokenizer.encode(str(text), add_special_tokens=True, truncation=True, padding=True) for text in val['tweet']]

train_labels = tf.convert_to_tensor(train['label'].tolist())
val_labels = tf.convert_to_tensor(val['label'].tolist())

train_tokens = tf.ragged.constant(train_tokens).to_tensor()
train_labels = tf.convert_to_tensor(train_labels)

train_dataset = tf.data.Dataset.from_tensor_slices((train_tokens, train_labels)).batch(32)

val_tokens = tf.ragged.constant(val_tokens).to_tensor()
val_labels = tf.convert_to_tensor(val_labels)

val_dataset = tf.data.Dataset.from_tensor_slices((val_tokens, val_labels)).batch(32)




optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model.fit(train_dataset, validation_data=val_dataset, epochs=3)

model.save('saved_model')

Epoch 1/3
100/100 [==============================] - 2915s 29s/step - loss: 0.7987 - accuracy: 0.6144 - val_loss: 0.7466 - val_accuracy: 0.6388
Epoch 2/3
100/100 [==============================] - 2857s 29s/step - loss: 0.6411 - accuracy: 0.7172 - val_loss: 0.8623 - val_accuracy: 0.6388
Epoch 3/3
100/100 [==============================] - 2946s 29s/step - loss: 0.6419 - accuracy: 0.6881 - val_loss: 0.8756 - val_accuracy: 0.6388


In [11]:
results = model.evaluate(val_dataset)
print(f"Validation accuracy: {results[1]:.2f}")

25/25 [==============================] - 199s 8s/step - loss: 0.8756 - accuracy: 0.6388
Validation accuracy: 0.64
